In [3]:
import pandas as pd
import glob
import os
import re

In [10]:
data = pd.read_pickle('/Users/gocchini/Desktop/paper_3/data/data_almost_complete.pkl')
london_postcodes = pd.read_csv('/Users/gocchini/Desktop/paper_3/data/london_postcodes-ons-postcodes-directory-feb22.csv')

In [4]:
data_main_shareholder = pd.concat([pd.read_excel(f, 'Results') for f in glob.glob('/Users/gocchini/Desktop/paper_3/data/FAME_2/*.xlsx')], ignore_index = True)

In [7]:
data_main_shareholder.to_pickle('/Users/gocchini/Desktop/paper_3/data/data_main_shareholder.pkl')

In [11]:
def data_processing(data, london_data): 
    data = data.loc[(data['Inactive'] == 'No')]
    data = data[data['Primary UK SIC (2007) code'].notna()]
    data = data[data['Primary trading address'].notna()]
    data = data.drop_duplicates(keep=False)
    data['postcode'] = data['Primary trading address'].apply(lambda x: x.split(',')[-1])
    data['postcode'] = data['postcode'].apply(lambda x: x[1:])
    london_postcodes = london_data['pcd'].tolist()
    london_df = data[data['postcode'].isin(london_postcodes)]
    london_df['Primary UK SIC (2007) code'] = london_df['Primary UK SIC (2007) code'].astype('string')
    return london_df 

In [12]:

london_df = data_processing(data , london_postcodes)

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_13240/2154885762.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  london_df['Primary UK SIC (2007) code'] = london_df['Primary UK SIC (2007) code'].astype('string')


In [13]:
london_intangibles = london_df[london_df['Primary UK SIC (2007) code'].str.contains(
    "^85|^92|^96|^59|^60|^61|^62|^63|^64|^66|^69|^70|^71|^72|^73|^74|^79|^82|^77|^78")]

In [20]:
london_intangible_main_shareholder = london_intangibles.merge(data_main_shareholder, left_on = 'Company name', right_on= 'Company name', how = 'inner')

In [27]:
with open ('/Users/gocchini/Desktop/paper_3/data/list-of-salutations-titles-honorifics.txt') as f: 
    lines = f.readlines()
honorifics = [line.strip() for line in lines]

In [52]:
def natural_entity_finder(entity, honorifics):
    if type(entity) != float:
        if any(honorific in entity for honorific in honorifics): 
            return 'natural person'
        else:
            return 'conglomerate'

In [53]:
london_intangible_main_shareholder['natural_entity_finder'] = london_intangible_main_shareholder.apply(lambda x: natural_entity_finder(x['PSC - Name'], honorifics), axis = 1)
london_intangible_main_shareholder = london_intangible_main_shareholder.loc[london_intangible_main_shareholder['natural_entity_finder'] ==  'natural person']

In [65]:
london_intangible_main_shareholder['Website'].to_pickle('/Users/gocchini/Desktop/paper_3/data/websites_paper_3.pkl')

In [7]:
text3 = pd.read_pickle('/Users/gocchini/Desktop/paper_1/scripts_paper_1/df_november_2022.pkl')
text3['actual_url'] = text3['url'].str.replace('http://', '')
text3['actual_url'] = text3['actual_url'].str.replace('https://', '')

In [8]:
london_data = london_intangibles.merge(text3, left_on = 'Website', right_on= 'actual_url')

In [9]:
london_data = london_data.loc[london_data['status']==True]

In [10]:
england_postcodes = pd.read_csv('/Users/gocchini/Downloads/England postcodes.csv')

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_33536/2786139566.py:1: DtypeWarning: Columns (18,29,47) have mixed types. Specify dtype option on import or set low_memory=False.
  england_postcodes = pd.read_csv('/Users/gocchini/Downloads/England postcodes.csv')


In [11]:
english_postcodes = england_postcodes['Postcode'].tolist()

In [15]:
def postcode_finder(text):
    postcodes = re.findall(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
    postcodes = [postcode[1] for postcode in postcodes]
    return postcodes 

In [16]:
london_data['text_postcode'] = london_data['text'].apply(postcode_finder)

In [17]:
def suspicious_postcodes_finder(postcode, postcode_in_text, english_postcodes):
    if len(postcode_in_text) > 0: 
        if postcode_in_text[0].isupper():
            if postcode_in_text[0] in english_postcodes:
                if postcode != postcode_in_text[0]:
                    return 'SUSPICIOUS!'
                else: 
                    return 'ok'

In [18]:
london_data['suspicious'] = london_data.apply(lambda x: suspicious_postcodes_finder(x['postcode'], x['text_postcode'], english_postcodes), axis = 1)